In [14]:
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import openpyxl as opx
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
import json

# Functions

In [3]:
# cast_timedelta('15m')

### Indices

In [8]:
def draw_block(x, y, w, h, text, colour):
    if not colour:
        # default white colour
        colour = 'FFFFFF'
    SHEET.merge_cells(start_row=y, start_column=x, end_row=y + h - 1, end_column=x + w - 1)
    merged_cell = SHEET.cell(row=y, column=x)  
    merged_cell.value = text
    merged_cell.alignment = Alignment(horizontal='center')
    merged_cell.fill = PatternFill("solid", fgColor=colour)
    
def draw_block_series(x, y, h, blocks):
    cur_x = x
    for w, text, colour in blocks:
        draw_block(cur_x, y, w, h, text, colour)
        cur_x += w
        
def draw_two_line_series(x, y, h, blocks1, blocks2):
    assert sum([block[0] for block in blocks1]) == sum([block[0] for block in blocks2])
    draw_block_series(x, y, h, blocks1)
    draw_block_series(x, y + h, h, blocks2)
    
    
def cast_w(duration):
    # one width is five minutes
    return int(duration / 5)

In [9]:
WB = opx.load_workbook(r'2020.10.06 pouring_template.xlsx')
SHEET = WB.worksheets[0]

draw_block(5, 10, 1, 1, 'a', '008000')

draw_block_series(8, 10, 1, 
                  [
                      [5, '60 налив', ''], 
                      [27, 'фиор ди латте  3,3 альче 80 юнитов  8000кг/11,5кг галактаза 5200', '']
                  ])

draw_two_line_series(10, 11, 1,
                  [
                      [5, '60 налив', ''], 
                      [27, 'фиор ди латте  3,3 альче 80 юнитов  8000кг/11,5кг галактаза 5200', '']
                  ],
                  [
                      [12, 'налив/внесение закваски', 'FFA500'],
                      [7, 'схватка', 'FFFF00'],
                      [8, 'резка/обсушка', '008000'],
                      [3, 'слив', 'FF0000'],
                      [2, '', '']
                  ])

draw_block(20, 20, cast_w(15), 1, 'a', '008000')


WB.save('output.xlsx')

# Simple algo


In [23]:
WB = opx.load_workbook(r'template.xlsx')
SHEET = WB.worksheets[0]
with open('template_config.yaml') as json_file:
    TEMPLATE_CONFIG = json.load(json_file)
    
with open('example.json') as json_file:
    example = json.load(json_file)

In [ ]:
def draw_block(x, y, w, h, text, colour):
    if not colour:
        colour = 'FFFFFF'
    SHEET.merge_cells(start_row=y, start_column=x, end_row=y + h - 1, end_column=x + w - 1)
    merged_cell = SHEET.cell(row=y, column=x)  
    merged_cell.value = text
    merged_cell.alignment = Alignment(horizontal='center')
    merged_cell.fill = PatternFill("solid", fgColor=colour)
    
def draw_block_series(x, y, h, blocks):
    cur_x = x
    for w, text, colour in blocks:
        draw_block(cur_x, y, w, h, text, colour)
        cur_x += w
        
def draw_two_line_series(x, y, h, blocks1, blocks2):
    assert sum([block[0] for block in blocks1]) == sum([block[0] for block in blocks2])
    draw_block_series(x, y, h, blocks1)
    draw_block_series(x, y + h, h, blocks2)
    
    
def cast_w(duration):
    # one width is five minutes
    return int(duration / 5)

# Object model

In [9]:
'''
    Маленькой блок, который обозначает один из процессов в сыроизготовителе
'''
class SimpleBlock:
    def __init__(self, name, duration, text, beg_time, raw_down=0, colour='FFFFFF'):
        self.name = name
        self.colour = colour
        self.text = text
        self.duration = duration
        self.beg_time = beg_time
        self.raw_down = raw_down
        
'''
    Процесс варки в сыроизготовителе.
    Задается имя, индекс варки, время начало и дополнительные параметры
'''        
class CheesePouring:
    def __init__(self, name, index, beg_time,
                 pouring_time=5 * 5, pouring_colour='FFFFFF', 
                 ferment_time=5 * 10, ferment_colour='FFA500',
                 hardening_time=5 * 6, hardening_colour='FFFF00', 
                 cutting_time=5 * 4, cutting_colour='008000',
                 pouring_off_time=5 * 3, pouring_off_colour='FF0000', 
                 extra_time=5 * 2, extra_colour='FFFFFF'):
        self.name = name
        self.index = index
        self.beg_time = beg_time
        
        self.pouring = SimpleBlock(name='Pouring', 
                                          colour=pouring_colour,
                                          duration=pouring_time,
                                          beg_time = beg_time,
                                          text='{} налив'.format(self.index))
        
        self.naming = SimpleBlock(name='Name',
                                         colour=pouring_colour,
                                         duration=ferment_time + hardening_time + cutting_time + pouring_off_time + extra_time - pouring_time,
                                         beg_time=beg_time + pouring_time,
                                         text=self.name)
        
        self.ferment = SimpleBlock(name='Ferment',
                                          colour=ferment_colour,
                                          duration=ferment_time,
                                          beg_time = beg_time,
                                          raw_down=1,
                                          text='налив/внесение закваски')
        
        self.hardening = SimpleBlock(name='Hardening',
                                            colour=hardening_colour,
                                            duration=hardening_time,
                                            beg_time = beg_time + ferment_time,
                                            raw_down=1,
                                            text='схватка')
        
        self.cutting = SimpleBlock(name='Cutting',
                                          colour=cutting_colour,
                                          duration=cutting_time,
                                          beg_time = beg_time + ferment_time + hardening_time,
                                          raw_down=1,
                                          text='резка/обсушка')
        
        self.pouring_off = SimpleBlock(name='Pouring off',
                                              colour=pouring_off_colour,
                                              duration=pouring_off_time,
                                              beg_time = beg_time + ferment_time + hardening_time + cutting_time,
                                              raw_down=1,
                                              text='слив')
        
        self.extra = SimpleBlock(name='Extra',
                                        colour=extra_colour,
                                        duration=extra_time,
                                        beg_time = beg_time + ferment_time + hardening_time + cutting_time + pouring_off_time,
                                        raw_down=1,
                                        text='')


'''
    Формирование процесса налива.
'''    
class CheesePouringExcel:
    def __init__(self, cheese_pouring):
        self.beg_time = 60
        self.beg_cell = 4
        self.period_time = 5
        
        self.cheese_pouring = cheese_pouring
        self.wb = opx.load_workbook(r'2020.10.06 pouring_template.xlsx')
        self.sheet = self.wb.worksheets[0]
    
    '''
        Сделать проверку данных (как минимум на пересечения)
    '''
    def prepare_data(self):
        pass
            
        
    '''
        Объединение ячеек в одну с нужным цветом и значением для одного блока
    '''
    def draw_pouring_block(self, pouring_block, cheese_maker_index):
        
        start_row = self.get_row_by_cheesemaker(cheese_maker_index) + pouring_block.raw_down
        start_column = self.get_column_by_time(pouring_block.beg_time)
        print(start_row, start_column)
        self.sheet.merge_cells(start_row=start_row,
                               start_column=start_column,
                               end_row=start_row,
                               end_column=start_column + pouring_block.duration / self.period_time - 1)
        print(start_row, start_column)
        merged_cell = self.sheet.cell(row=start_row, column=start_column)  
        merged_cell.value = pouring_block.text
        merged_cell.alignment = Alignment(horizontal='center')
        merged_cell.fill = PatternFill("solid", fgColor=pouring_block.colour)
        
    '''
        Формирование полного блока
    '''    
    def draw_cheese_pouring(self, cheese_pouring, cheese_maker_index):
        self.draw_pouring_block(cheese_pouring.pouring, cheese_maker_index)
        self.draw_pouring_block(cheese_pouring.naming, cheese_maker_index)
        self.draw_pouring_block(cheese_pouring.ferment, cheese_maker_index)
        self.draw_pouring_block(cheese_pouring.hardening, cheese_maker_index)
        self.draw_pouring_block(cheese_pouring.cutting, cheese_maker_index)
        self.draw_pouring_block(cheese_pouring.pouring_off, cheese_maker_index)
        self.draw_pouring_block(cheese_pouring.extra, cheese_maker_index)
                        
    '''
        Индекс стоблца по времени
    '''        
    def get_column_by_time(self, time):
        return (time - self.beg_time) / self.period_time + self.beg_cell
    
    
    '''
        Индекс строки по номеру сыроизготовителя
        КОСТЫЛЬ
    '''
    def get_row_by_cheesemaker(self, index):
        if index == 0: return 6
        if index == 1: return 9
        if index == 2: return 15
        if index == 3: return 18
#         return self.cheese_makers[index].row
    
    
    '''
        Сохранение excel
    '''
    def save_output(self):
        self.wb.save('output.xlsx')
        
#         merge_cells(start_row=2, start_column=1, end_row=4, end_column=4)
        #         self.sheet.cell(row=2, column=2).value = 2    

## Test